In [1]:
import pybullet as p
import sys
sys.path.append("../install/lib")
import moveit_noros as moveit
import pybullet_data
import time
import numpy as np
import threading

In [2]:
def getTransform(bulletHardware, planning_scene, link_name):
    moveit.syncBulletState(bulletHardware, planning_scene)
    return planning_scene.getCurrentStateNonConst().getGlobalLinkTransform(link_name)

In [3]:
print("== Initialize Pybullet == ")
p.connect(p.GUI_SERVER)
p.setRealTimeSimulation(1)

== Initialize Pybullet == 


In [4]:
p.setGravity(0,0,-10)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
planeId = p.loadURDF("plane.urdf")

p.setAdditionalSearchPath("../resources")
startPos = [0,0,0]
startOrientation = p.getQuaternionFromEuler([0, 0, 0])
boxId = p.loadURDF("pr2.urdf", startPos, startOrientation, useFixedBase=1)

print("== Initialize moveit ==")
robot_loader = moveit.createRobotModelLoaderFromFile("../resources/pr2.urdf", "../resources/pr2_config/pr2.srdf")
kinematic_solver_loader = moveit.createKinematicsLoaderFromFile("../resources/pr2_config/kinematics.yaml")
robot_loader.loadKinematicsSolvers(kinematic_solver_loader)
robot_model = robot_loader.getModel()

pconfig = moveit.createPlannerConfigurationFromFile("../resources/pr2_config/ompl_planning.yaml")
planner = moveit.PlannerManager(robot_model, pconfig)

bullet_hardware = moveit.PybulletHardware(p, boxId)
planning_scene = robot_loader.newPlanningScene()
scene_helper = moveit.PlanningSceneHelper(bullet_hardware, planning_scene)

== Initialize moveit ==


In [5]:
PLANNING_GROUP = "left_arm"
joint_model_group = robot_model.getJointModelGroup(PLANNING_GROUP)

base_frame = joint_model_group.base_frame
tip_frame = joint_model_group.tip_frame

In [7]:
link_transform = scene_helper.linkTransform(tip_frame)
base_transform = scene_helper.linkTransform(base_frame, False)
relative_transform = scene_helper.linkRelativeTransform(tip_frame, base_frame, False)

In [8]:
link_transform

<EigenAffine3d from Eigen::Affine3d>
Translation:
0.759038
0.193049
0.659922
Rotation:
  0.983404  0.0668989   0.168643
0.00613946   0.916731  -0.399458
 -0.181324   0.393864   0.901107

In [9]:
base_transform

<EigenAffine3d from Eigen::Affine3d>
Translation:
   -0.05
       0
0.790628
Rotation:
1 0 0
0 1 0
0 0 1

In [10]:
relative_transform

<EigenAffine3d from Eigen::Affine3d>
Translation:
 0.809038
 0.193049
-0.130705
Rotation:
  0.983404  0.0668989   0.168643
0.00613946   0.916731  -0.399458
 -0.181324   0.393864   0.901107

In [12]:
base_transform.inverse * link_transform

<EigenAffine3d from Eigen::Affine3d>
Translation:
 0.809038
 0.193049
-0.130705
Rotation:
  0.983404  0.0668989   0.168643
0.00613946   0.916731  -0.399458
 -0.181324   0.393864   0.901107

In [11]:
req = moveit.MotionPlanRequest(PLANNING_GROUP)
pose = moveit.PoseStamped(base_frame, end_transform.translation, end_transform.quaternion.value)
pose_goal = moveit.constructGoalConstraints(tip_link, pose, 0.01, 0.01)
req.addGoal(pose_goal)

In [12]:
context = planner.getPlanningContext(planning_scene, req)
response = context.solve()
response.trajectory.computeTimeStamps()

AttributeError: 'NoneType' object has no attribute 'computeTimeStamps'

In [13]:
end_transform.translation

array([0.6259753 , 0.17461295, 1.1247314 ])

In [10]:
spline = moveit.computeSpline(response.trajectory)

In [16]:
for i in range(len(spline.tip_transforms)-1):
    p.addUserDebugLine(spline.tip_transforms[i].translation, spline.tip_transforms[i+1].translation)

In [21]:
p.addUserDebugLine(np.array([0.629203, 0.142431, 0.705641]), np.array([0.562863, 0.367344, 1.18433]), [1,0,0], 5)

36

In [12]:
transform

<EigenAffine3d from Eigen::Affine3d>
Translation:
0.626821
0.222088
 1.19892
Rotation:
   0.67486  -0.642299   -0.36334
0.00100742   0.493169  -0.869933
  0.737945   0.586717   0.333467

In [13]:
transform.quaternion

<Eigen::Quaterniond at 0x560edd06d390>
[ 0.460495, -0.348153, 0.20337, 0.790806]

In [14]:
transform.quaternion.value

array([ 0.46049543, -0.34815289,  0.20337041,  0.79080592])

In [15]:
transform.quaternion.matrix

array([[ 0.67486009, -0.64229867, -0.36334045],
       [ 0.00100742,  0.49316888, -0.86993301],
       [ 0.73794502,  0.58671703,  0.33346705]])

In [17]:
base_frame

'/odom_combined'

In [11]:
transform.quaternion[0]

TypeError: 'moveit_noros.EigenQuaterniond' object is not subscriptable

In [11]:
robot_model.getJointModelGroup

<bound method PyCapsule.getJointModelGroup of <moveit_noros.RobotModel object at 0x7f6292ae1630>>

In [12]:
moveit.createKinematicsLoaderFromFile

<function moveit_noros.PyCapsule.createKinematicsLoaderFromFile>